In [1]:
!pip install faiss-cpu langchain-community "sentence-transformers" "faiss-cpu" transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.9/469.9 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56

In [2]:
!pip install addict easydict

In [3]:
import json
import time
import torch
import argparse
import os
import os
import warnings
from typing import Dict, List, Any, Tuple
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
import re

# Suppress warnings
warnings.filterwarnings('ignore')
os.environ['TRANSFORMERS_VERBOSITY'] = 'error'

# Mock GraphRAG implementation (since actual graphrag library may not be available)
class MockGraphRAG:
    def __init__(self, kg_data: Dict, embedding_model, rerank_model):
        self.entities = {e['id']: e for e in kg_data['entities']}
        self.triplets = kg_data['triplets']
        self.embedding_model = embedding_model
        self.rerank_model = rerank_model
        
    def query(self, query_text: str, top_k: int = 5) -> str:
        """Retrieve relevant context from knowledge graph"""
        # Simple retrieval: match keywords in query with entity descriptions
        contexts = []
        query_lower = query_text.lower()
        
        for entity_id, entity in self.entities.items():
            desc = entity.get('description', '').lower()
            original = entity.get('original_text', '').lower()
            
            # Score based on keyword overlap
            score = 0
            for word in query_lower.split():
                if len(word) > 2:
                    if word in desc:
                        score += 2
                    if word in original:
                        score += 1
            
            if score > 0:
                contexts.append((score, entity['original_text'][:500]))
        
        # Sort by score and return top contexts
        contexts.sort(reverse=True, key=lambda x: x[0])
        retrieved = "\n\n".join([ctx for _, ctx in contexts[:top_k]])
        return retrieved if retrieved else "No relevant information found."


class GraphRAGEvaluator:
    def __init__(self, kg_data: Dict, questions_data: Dict, 
                 llm_model_name: str = "deepseek-ai/DeepSeek-R1-0528-Qwen3-8B",
                 embed_model_name: str = "Qwen/Qwen3-Embedding-0.6B",
                 rerank_model_name: str = "Qwen/Qwen3-Reranker-0.6B"):
        
        self.kg_data = kg_data
        self.questions_data = questions_data
        self.gpu_count = torch.cuda.device_count() if torch.cuda.is_available() else 0
        
        print("Loading models...")
        print(f"LLM: {llm_model_name}")
        print(f"Embedding: {embed_model_name}")
        print(f"Rerank: {rerank_model_name}")
        
        # Load models (simplified for mock implementation)
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(llm_model_name, trust_remote_code=True)
            self.llm = AutoModelForCausalLM.from_pretrained(
                llm_model_name, 
                trust_remote_code=True,
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                device_map="auto" if torch.cuda.is_available() else None
            )
        except:
            print("Warning: Could not load actual models. Using mock implementation.")
            self.tokenizer = None
            self.llm = None
        
        # Initialize GraphRAG
        self.graph_rag = MockGraphRAG(kg_data, None, None)
        
    def parse_question(self, question_text: str) -> str:
        """Parse question into query components"""
        # Simple entity extraction
        entities = []
        for entity_id, entity in self.graph_rag.entities.items():
            for label in entity.get('label', []):
                if label.lower() in question_text.lower():
                    entities.append(label)
        
        return question_text + " " + " ".join(entities)
    
    def retrieve_context(self, question: str) -> Tuple[str, int]:
        """Retrieve relevant context from knowledge graph"""
        parsed_query = self.parse_question(question)
        context = self.graph_rag.query(parsed_query)
        return context, len(context)
    
    def generate_answer(self, question: str, context: str, options: List[str]) -> Tuple[str, int]:
        """Generate answer using LLM"""
        # Format options
        options_text = "\n".join([f"{chr(65+i)}. {opt}" for i, opt in enumerate(options)])
        
        prompt = f"""Dựa vào thông tin sau:
{context}

Câu hỏi: {question}

Các lựa chọn:
{options_text}

Hãy chọn đáp án đúng nhất và chỉ trả lời bằng chữ cái (A, B, C, hoặc D). Không giải thích thêm.

Đáp án:"""
        
        if self.llm and self.tokenizer:
            try:
                inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048)
                if torch.cuda.is_available():
                    inputs = {k: v.cuda() for k, v in inputs.items()}
                
                with torch.no_grad():
                    outputs = self.llm.generate(
                        **inputs,
                        max_new_tokens=10,
                        do_sample=False
                    )
                
                response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
                response = response[len(prompt):].strip()
            except Exception as e:
                print(f"Generation error: {e}")
                response = "A"
        else:
            # Mock response based on context matching
            response = self._mock_answer(question, context, options)
        
        return response, len(response)
    
    def _mock_answer(self, question: str, context: str, options: List[str]) -> str:
        """Mock answer selection based on keyword matching"""
        scores = []
        context_lower = context.lower()
        
        for opt in options:
            opt_lower = opt.lower()
            score = sum(1 for word in opt_lower.split() if len(word) > 3 and word in context_lower)
            scores.append(score)
        
        max_score = max(scores) if scores else 0
        if max_score > 0:
            return chr(65 + scores.index(max_score))
        return "A"
    
    def extract_answer(self, response: str, question_type: str = "multiple_choice") -> Tuple[str, float, str]:
        """Extract answer from model response"""
        # Try to find letter answer (A, B, C, D)
        response_upper = response.upper().strip()
        
        # Direct match
        if len(response_upper) == 1 and response_upper in ['A', 'B', 'C', 'D']:
            return response_upper, 1.0, "direct_match"
        
        # Pattern matching
        patterns = [
            r'\b([ABCD])\b',
            r'đáp án[:\s]+([ABCD])',
            r'chọn[:\s]+([ABCD])',
            r'^([ABCD])[.\s]',
        ]
        
        for pattern in patterns:
            match = re.search(pattern, response_upper)
            if match:
                return match.group(1), 0.8, "pattern_match"
        
        # For true/false, check for keywords
        if question_type == "true_false":
            if any(word in response.lower() for word in ['đúng', 'correct', 'true']):
                return 'A', 0.6, "keyword_match"
            if any(word in response.lower() for word in ['sai', 'wrong', 'false']):
                return 'B', 0.6, "keyword_match"
        
        return "[NO PREDICTION]", 0.0, "no_match"
    
    def extract_statement_from_tf_question(self, question: str) -> Tuple[str, str]:
        """Extract given context and statement from true/false question"""
        # Split question into context and statement
        parts = question.split('Phát biểu sau:')
        if len(parts) == 2:
            given_context = parts[0].replace('Đọc đoạn tư liệu sau đây:', '').strip().strip('"')
            statement_part = parts[1].strip().strip('"')
            # Remove "là đúng hay sai" at the end
            statement = statement_part.replace('là đúng hay sai', '').strip()
            return given_context, statement
        return "", question
    
    def evaluate_multiple_choice(self, num_questions: int = None) -> Dict:
        """Evaluate multiple choice questions"""
        questions = self.questions_data['multiple_choice']
        if num_questions:
            questions = questions[:num_questions]
        
        results = {
            'stats': {'correct': 0, 'wrong': 0, 'no_prediction': 0, 'total': len(questions), 'accuracy': 0.0},
            'questions': []
        }
        
        print(f"\nEvaluating {len(questions)} Multiple Choice questions...")
        
        for idx, q_data in enumerate(questions, 1):
            if idx % 10 == 0 or idx == 1:
                print(f"  Progress: {idx}/{len(questions)}")
            
            question_text = q_data['question']
            options_data = q_data['options']
            option_texts = [opt['answer'] for opt in options_data]
            
            # Find correct answer safely
            correct_idx = None
            for i, opt in enumerate(options_data):
                if opt.get('isCorrect', False):
                    correct_idx = i
                    break
            
            if correct_idx is None:
                print(f"  Warning: Question {idx} has no correct answer marked. Skipping.")
                continue
            
            correct_answer = chr(65 + correct_idx)
            
            # Retrieve context
            context, ctx_len = self.retrieve_context(question_text)
            
            # Generate answer
            response, resp_len = self.generate_answer(question_text, context, option_texts)
            
            # Extract answer
            predicted_answer, confidence, method = self.extract_answer(response, "multiple_choice")
            
            # Check correctness
            is_correct = (predicted_answer == correct_answer)
            if predicted_answer == "[NO PREDICTION]":
                results['stats']['no_prediction'] += 1
            elif is_correct:
                results['stats']['correct'] += 1
            else:
                results['stats']['wrong'] += 1
            
            # Build options with markers
            options_list = []
            for i, opt_text in enumerate(option_texts):
                letter = chr(65 + i)
                is_opt_correct = (i == correct_idx)
                is_predicted = (letter == predicted_answer)
                marker = ""
                if is_opt_correct and is_predicted:
                    marker = "(correct, predicted)"
                elif is_opt_correct:
                    marker = "(correct)"
                elif is_predicted:
                    marker = "(predicted)"
                
                options_list.append({
                    'letter': letter,
                    'text': opt_text,
                    'is_correct': is_opt_correct,
                    'is_predicted': is_predicted,
                    'marker': marker
                })
            
            results['questions'].append({
                'id': idx,
                'question': question_text,
                'options': options_list,
                'predicted_answer': predicted_answer,
                'correct_answer': correct_answer,
                'is_correct': is_correct,
                'retrieval': {'context': context, 'context_length': ctx_len},
                'generation': {'full_response': response, 'response_length': resp_len},
                'extraction': {'method': method, 'confidence': confidence}
            })
        
        results['stats']['accuracy'] = (results['stats']['correct'] / results['stats']['total'] * 100) if results['stats']['total'] > 0 else 0
        return results
    
    def evaluate_true_false(self, num_questions: int = None) -> Dict:
        """Evaluate true/false questions"""
        questions = self.questions_data['true_false']
        if num_questions:
            questions = questions[:num_questions]
        
        results = {
            'stats': {'correct': 0, 'wrong': 0, 'no_prediction': 0, 'total': len(questions), 'accuracy': 0.0},
            'questions': []
        }
        
        print(f"\nEvaluating {len(questions)} True/False questions...")
        
        for idx, q_data in enumerate(questions, 1):
            if idx % 10 == 0 or idx == 1:
                print(f"  Progress: {idx}/{len(questions)}")
            
            question_text = q_data['question']
            options_data = q_data['options']
            option_texts = [opt['answer'] for opt in options_data]
            
            # Find correct answer safely
            correct_idx = None
            for i, opt in enumerate(options_data):
                if opt.get('isCorrect', False):
                    correct_idx = i
                    break
            
            if correct_idx is None:
                print(f"  Warning: Question {idx} has no correct answer marked. Skipping.")
                continue
            
            correct_answer = 'A' if correct_idx == 0 else 'B'
            
            # Extract given context and statement
            given_context, statement = self.extract_statement_from_tf_question(question_text)
            
            # Retrieve context
            context, ctx_len = self.retrieve_context(statement)
            
            # Generate answer
            response, resp_len = self.generate_answer(statement, context, option_texts)
            
            # Extract answer
            predicted_answer, confidence, method = self.extract_answer(response, "true_false")
            
            # Check correctness
            is_correct = (predicted_answer == correct_answer)
            if predicted_answer == "[NO PREDICTION]":
                results['stats']['no_prediction'] += 1
            elif is_correct:
                results['stats']['correct'] += 1
            else:
                results['stats']['wrong'] += 1
            
            # Build options with markers
            options_list = []
            for i, opt_text in enumerate(option_texts):
                letter = 'A' if i == 0 else 'B'
                is_opt_correct = (i == correct_idx)
                is_predicted = (letter == predicted_answer)
                marker = ""
                if is_opt_correct and is_predicted:
                    marker = "(correct, predicted)"
                elif is_opt_correct:
                    marker = "(correct)"
                elif is_predicted:
                    marker = "(predicted)"
                
                options_list.append({
                    'letter': letter,
                    'text': opt_text,
                    'is_correct': is_opt_correct,
                    'is_predicted': is_predicted,
                    'marker': marker
                })
            
            results['questions'].append({
                'id': idx,
                'question': question_text,
                'statement': statement,
                'given_context': given_context,
                'options': options_list,
                'predicted_answer': predicted_answer,
                'correct_answer': correct_answer,
                'is_correct': is_correct,
                'retrieval': {'context': context, 'context_length': ctx_len},
                'generation': {'full_response': response, 'response_length': resp_len},
                'extraction': {'method': method, 'confidence': confidence}
            })
        
        results['stats']['accuracy'] = (results['stats']['correct'] / results['stats']['total'] * 100) if results['stats']['total'] > 0 else 0
        return results
    
    def evaluate_all(self, num_mc: int = None, num_tf: int = None) -> Dict:
        """Evaluate all questions"""
        print("\n" + "="*60)
        print("STARTING EVALUATION")
        print("="*60)
        
        start_time = time.time()
        
        if torch.cuda.is_available():
            for i in range(self.gpu_count):
                torch.cuda.reset_peak_memory_stats(i)
        
        mc_results = self.evaluate_multiple_choice(num_mc)
        tf_results = self.evaluate_true_false(num_tf)
        
        elapsed = time.time() - start_time
        
        total_correct = mc_results['stats']['correct'] + tf_results['stats']['correct']
        total_questions = mc_results['stats']['total'] + tf_results['stats']['total']
        total_no_pred = mc_results['stats']['no_prediction'] + tf_results['stats']['no_prediction']
        overall_accuracy = (total_correct / total_questions * 100) if total_questions > 0 else 0
        prediction_rate = ((total_questions - total_no_pred) / total_questions * 100) if total_questions > 0 else 0
        
        results = {
            'multiple_choice': mc_results,
            'true_false': tf_results,
            'overall': {
                'total_correct': total_correct,
                'total_questions': total_questions,
                'overall_accuracy': overall_accuracy,
                'elapsed_time': f"{elapsed:.2f}s",
                'total_no_prediction': total_no_pred,
                'prediction_rate': prediction_rate
            }
        }
        
        print("\n" + "="*60)
        print("EVALUATION COMPLETED")
        print("="*60)
        
        return results
    
    def print_summary(self, results: Dict):
        """Print evaluation summary"""
        print(f"\n{'='*60}")
        print(f"{'FINAL RESULTS':^60}")
        print(f"{'='*60}")
        
        mc = results['multiple_choice']['stats']
        tf = results['true_false']['stats']
        ov = results['overall']
        
        print(f"\nMULTIPLE CHOICE: {mc['correct']}/{mc['total']} = {mc['accuracy']:.1f}%")
        print(f"  Wrong: {mc.get('wrong',0)}  No Answer: {mc.get('no_prediction',0)}")
        print(f"TRUE/FALSE:      {tf['correct']}/{tf['total']} = {tf['accuracy']:.1f}%")
        print(f"  Wrong: {tf.get('wrong',0)}  No Answer: {tf.get('no_prediction',0)}")
        print(f"\nOVERALL:         {ov['total_correct']}/{ov['total_questions']} = {ov['overall_accuracy']:.1f}%")
        print(f"TIME:            {ov['elapsed_time']}")
        print(f"{'='*60}\n")
        
        if torch.cuda.is_available():
            total_mem = sum(torch.cuda.max_memory_allocated(i) for i in range(self.gpu_count))
            print(f"Peak GPU Memory: {total_mem / 1024**3:.2f} GB")

    def generate_html_report(self, results: Dict, output_path: str):
        """Tạo báo cáo HTML dễ đọc"""
        html = f"""<!DOCTYPE html>
<html lang="vi">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>RAG Evaluation Report</title>
    <style>
        body {{ font-family: 'Segoe UI', Arial, sans-serif; margin: 20px; background: #f5f5f5; }}
        .header {{ background: #2c3e50; color: white; padding: 20px; border-radius: 8px; margin-bottom: 20px; }}
        .stats {{ display: grid; grid-template-columns: repeat(auto-fit, minmax(200px, 1fr)); gap: 15px; margin-bottom: 30px; }}
        .stat-card {{ background: white; padding: 20px; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }}
        .stat-value {{ font-size: 32px; font-weight: bold; color: #3498db; }}
        .stat-label {{ color: #7f8c8d; margin-top: 5px; }}
        .question-card {{ background: white; padding: 20px; margin-bottom: 20px; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); border-left: 4px solid #ddd; }}
        .question-card.correct {{ border-left-color: #27ae60; }}
        .question-card.wrong {{ border-left-color: #e74c3c; }}
        .question-card.no-answer {{ border-left-color: #f39c12; }}
        .question-header {{ display: flex; justify-content: space-between; align-items: center; margin-bottom: 15px; }}
        .question-id {{ font-weight: bold; color: #2c3e50; }}
        .status-badge {{ padding: 5px 15px; border-radius: 20px; font-size: 12px; font-weight: bold; }}
        .status-badge.correct {{ background: #27ae60; color: white; }}
        .status-badge.wrong {{ background: #e74c3c; color: white; }}
        .status-badge.no-answer {{ background: #f39c12; color: white; }}
        .question-text {{ font-size: 16px; margin-bottom: 15px; line-height: 1.6; }}
        .options {{ margin: 15px 0; }}
        .option {{ padding: 10px; margin: 5px 0; border-radius: 5px; background: #ecf0f1; }}
        .option.correct {{ background: #d5f4e6; border-left: 3px solid #27ae60; }}
        .option.predicted {{ background: #fadbd8; border-left: 3px solid #e74c3c; }}
        .option.correct.predicted {{ background: #d5f4e6; border-left: 3px solid #27ae60; }}
        .section-title {{ font-size: 24px; font-weight: bold; margin: 30px 0 15px 0; padding-bottom: 10px; border-bottom: 2px solid #3498db; }}
        .context-box {{ background: #ecf0f1; padding: 15px; border-radius: 5px; margin: 10px 0; max-height: 200px; overflow-y: auto; font-size: 14px; line-height: 1.6; }}
        .response-box {{ background: #fff3cd; padding: 15px; border-radius: 5px; margin: 10px 0; max-height: 150px; overflow-y: auto; font-size: 14px; }}
        .metadata {{ display: grid; grid-template-columns: repeat(auto-fit, minmax(150px, 1fr)); gap: 10px; margin-top: 15px; font-size: 13px; color: #7f8c8d; }}
        .metadata-item {{ background: #f8f9fa; padding: 8px; border-radius: 4px; }}
        .toggle-btn {{ cursor: pointer; color: #3498db; font-size: 13px; margin-top: 10px; user-select: none; }}
        .collapsible {{ display: none; }}
        .collapsible.show {{ display: block; }}
    </style>
</head>
<body>
    <div class="header">
        <h1>RAG System Evaluation Report</h1>
        <p>Generated: {time.strftime('%Y-%m-%d %H:%M:%S')}</p>
    </div>
    
    <div class="stats">
        <div class="stat-card">
            <div class="stat-value">{results['overall']['overall_accuracy']:.1f}%</div>
            <div class="stat-label">Overall Accuracy</div>
        </div>
        <div class="stat-card">
            <div class="stat-value">{results['overall']['total_correct']}/{results['overall']['total_questions']}</div>
            <div class="stat-label">Correct Answers</div>
        </div>
        <div class="stat-card">
            <div class="stat-value">{results['overall'].get('prediction_rate', 0):.1f}%</div>
            <div class="stat-label">Prediction Rate</div>
        </div>
        <div class="stat-card">
            <div class="stat-value">{results['overall'].get('total_no_prediction', 0)}</div>
            <div class="stat-label">No Predictions</div>
        </div>
    </div>
"""
        
        # Multiple Choice Section
        mc_stats = results['multiple_choice']['stats']
        html += f"""
    <div class="section-title">Multiple Choice Questions ({mc_stats.get('total',0)})</div>
    <div class="stats">
        <div class="stat-card">
            <div class="stat-value">{mc_stats.get('accuracy',0):.1f}%</div>
            <div class="stat-label">Accuracy</div>
        </div>
        <div class="stat-card">
            <div class="stat-value">{mc_stats.get('correct',0)}</div>
            <div class="stat-label">Correct</div>
        </div>
        <div class="stat-card">
            <div class="stat-value">{mc_stats.get('wrong',0)}</div>
            <div class="stat-label">Wrong</div>
        </div>
        <div class="stat-card">
            <div class="stat-value">{mc_stats.get('no_prediction',0)}</div>
            <div class="stat-label">No Answer</div>
        </div>
    </div>
"""
        for q in results['multiple_choice']['questions']:
            status_class = 'correct' if q['is_correct'] else ('no-answer' if q['predicted_answer'] == '[NO PREDICTION]' else 'wrong')
            status_text = 'CORRECT' if q['is_correct'] else ('NO ANSWER' if q['predicted_answer'] == '[NO PREDICTION]' else 'WRONG')
            
            html += f"""
    <div class="question-card {status_class}">
        <div class="question-header">
            <span class="question-id">Question #{q['id']}</span>
            <span class="status-badge {status_class}">{status_text}</span>
        </div>
        <div class="question-text"><strong>Q:</strong> {q['question']}</div>
        
        <div class="options">
"""
            for opt in q.get('options', []):
                opt_class = ''
                if opt['is_correct'] and opt['is_predicted']:
                    opt_class = 'correct predicted'
                elif opt['is_correct']:
                    opt_class = 'correct'
                elif opt['is_predicted']:
                    opt_class = 'predicted'
                
                marker = opt.get('marker', '')
                html += f"""            <div class="option {opt_class}">{opt['letter']}. {opt['text']} {marker}</div>\n"""
            
            html += f"""
        </div>
        
        <div style="margin-top: 15px;">
            <strong>Predicted:</strong> {q['predicted_answer']}<br>
            <strong>Correct:</strong> {q['correct_answer']}
        </div>
        
        <div class="toggle-btn" onclick="toggleCollapsible('mc-{q['id']}')">Show Details</div>
        <div id="mc-{q['id']}" class="collapsible">
            <div class="metadata">
                <div class="metadata-item">
                    <strong>Extraction Method:</strong><br>{q.get('extraction', {}).get('method', 'N/A')}
                </div>
                <div class="metadata-item">
                    <strong>Confidence:</strong><br>{q.get('extraction', {}).get('confidence', 0):.2f}
                </div>
                <div class="metadata-item">
                    <strong>Context Length:</strong><br>{q.get('retrieval', {}).get('context_length', 0)} chars
                </div>
                <div class="metadata-item">
                    <strong>Response Length:</strong><br>{q.get('generation', {}).get('response_length', 0)} chars
                </div>
            </div>
            
            <h4>Retrieved Context:</h4>
            <div class="context-box">{q.get('retrieval', {}).get('context', 'N/A')}</div>
            
            <h4>Model Response:</h4>
            <div class="response-box">{q.get('generation', {}).get('full_response', 'N/A')}</div>
        </div>
    </div>
"""
        
        # True/False Section
        tf_stats = results['true_false']['stats']
        html += f"""
    <div class="section-title">True/False Questions ({tf_stats.get('total',0)})</div>
    <div class="stats">
        <div class="stat-card">
            <div class="stat-value">{tf_stats.get('accuracy',0):.1f}%</div>
            <div class="stat-label">Accuracy</div>
        </div>
        <div class="stat-card">
            <div class="stat-value">{tf_stats.get('correct',0)}</div>
            <div class="stat-label">Correct</div>
        </div>
        <div class="stat-card">
            <div class="stat-value">{tf_stats.get('wrong',0)}</div>
            <div class="stat-label">Wrong</div>
        </div>
        <div class="stat-card">
            <div class="stat-value">{tf_stats.get('no_prediction',0)}</div>
            <div class="stat-label">No Answer</div>
        </div>
    </div>
"""
        for q in results['true_false']['questions']:
            status_class = 'correct' if q['is_correct'] else ('no-answer' if q['predicted_answer'] == '[NO PREDICTION]' else 'wrong')
            status_text = 'CORRECT' if q['is_correct'] else ('NO ANSWER' if q['predicted_answer'] == '[NO PREDICTION]' else 'WRONG')
            
            html += f"""
    <div class="question-card {status_class}">
        <div class="question-header">
            <span class="question-id">Question #{q['id']}</span>
            <span class="status-badge {status_class}">{status_text}</span>
        </div>
        <div class="question-text"><strong>Statement:</strong> {q.get('statement', 'N/A')}</div>
        
        <div class="options">
"""
            for opt in q.get('options', []):
                opt_class = ''
                if opt['is_correct'] and opt['is_predicted']:
                    opt_class = 'correct predicted'
                elif opt['is_correct']:
                    opt_class = 'correct'
                elif opt['is_predicted']:
                    opt_class = 'predicted'
                
                marker = opt.get('marker', '')
                html += f"""            <div class="option {opt_class}">{opt['letter']}. {opt['text']} {marker}</div>\n"""
            
            html += f"""
        </div>
        
        <div style="margin-top: 15px;">
            <strong>Predicted:</strong> {q['predicted_answer']}<br>
            <strong>Correct:</strong> {q['correct_answer']}
        </div>
        
        <div class="toggle-btn" onclick="toggleCollapsible('tf-{q['id']}')">Show Details</div>
        <div id="tf-{q['id']}" class="collapsible">
            <div class="metadata">
                <div class="metadata-item">
                    <strong>Extraction Method:</strong><br>{q.get('extraction', {}).get('method', 'N/A')}
                </div>
                <div class="metadata-item">
                    <strong>Confidence:</strong><br>{q.get('extraction', {}).get('confidence', 0):.2f}
                </div>
                <div class="metadata-item">
                    <strong>Context Length:</strong><br>{q.get('retrieval', {}).get('context_length', 0)} chars
                </div>
                <div class="metadata-item">
                    <strong>Response Length:</strong><br>{q.get('generation', {}).get('response_length', 0)} chars
                </div>
            </div>
            
            <h4>Given Context (from question):</h4>
            <div class="context-box">{q.get('given_context', 'N/A')}</div>
            
            <h4>Retrieved Context:</h4>
            <div class="context-box">{q.get('retrieval', {}).get('context', 'N/A')}</div>
            
            <h4>Model Response:</h4>
            <div class="response-box">{q.get('generation', {}).get('full_response', 'N/A')}</div>
        </div>
    </div>
"""
        
        html += """
    <script>
        function toggleCollapsible(id) {
            const element = document.getElementById(id);
            element.classList.toggle('show');
        }
    </script>
</body>
</html>
"""
        
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(html)
        
        print(f"HTML report saved to: {output_path}")


def load_json_file(filepath: str) -> Dict:
    """Load JSON file"""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"Error: File not found: {filepath}")
        raise
    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON in {filepath}: {e}")
        raise


def main(kg_file='/kaggle/input/kltn-1/graph_documents_v3.json',
         questions_file='/kaggle/input/kltn-1/question_1000.json',
         num_mc=None,
         num_tf=None,
         output='report.html'):
    """Main execution function
    
    Args:
        kg_file: Path to knowledge graph JSON file
        questions_file: Path to questions JSON file
        num_mc: Number of multiple choice questions to evaluate (None = all)
        num_tf: Number of true/false questions to evaluate (None = all)
        output: Output HTML report path
    """
    
    print("="*60)
    print("GraphRAG Question Answering System")
    print("="*60)
    print(f"Loading data from files...")
    print(f"  KG file: {kg_file}")
    print(f"  Questions file: {questions_file}")
    
    # Load data from files
    try:
        kg_data = load_json_file(kg_file)
        questions_data = load_json_file(questions_file)
        print(f"  Loaded {len(kg_data.get('entities', []))} entities")
        print(f"  Loaded {len(kg_data.get('triplets', []))} triplets")
        print(f"  Loaded {len(questions_data.get('multiple_choice', []))} multiple choice questions")
        print(f"  Loaded {len(questions_data.get('true_false', []))} true/false questions")
    except Exception as e:
        print(f"Failed to load data: {e}")
        return
    

    
    print("="*60)
    print("GraphRAG Question Answering System")
    print("="*60)
    print(f"Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")
    if torch.cuda.is_available():
        print(f"GPU Count: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
    print("="*60)
    
    # Initialize evaluator
    evaluator = GraphRAGEvaluator(kg_data, questions_data)
    
    # Run evaluation
    print("\nStarting evaluation...")
    results = evaluator.evaluate_all(num_mc=num_mc, num_tf=num_tf)
    
    # Print summary
    evaluator.print_summary(results)
    
    # Generate HTML report
    evaluator.generate_html_report(results, output)
    
    print("\nEvaluation complete!")
    
    return results


def main_cli():
    """Command-line interface wrapper for main()"""
    parser = argparse.ArgumentParser(description='GraphRAG Question Answering Evaluation')
    parser.add_argument('--kg-file', type=str, 
                        default='/kaggle/input/kltn-1/graph_documents_v3.json',
                        help='Path to knowledge graph JSON file')
    parser.add_argument('--questions-file', type=str,
                        default='/kaggle/input/kltn-1/question_1000.json',
                        help='Path to questions JSON file')
    parser.add_argument('--num-mc', type=int, default=None, 
                        help='Number of multiple choice questions to evaluate (default: all)')
    parser.add_argument('--num-tf', type=int, default=None,
                        help='Number of true/false questions to evaluate (default: all)')
    parser.add_argument('--output', type=str, default='report.html',
                        help='Output HTML report path (default: report.html)')
    
    # Use parse_known_args to ignore Jupyter kernel arguments
    args, _ = parser.parse_known_args()
    
    return main(
        kg_file=args.kg_file,
        questions_file=args.questions_file,
        num_mc=args.num_mc,
        num_tf=args.num_tf,
        output=args.output
    )


if __name__ == "__main__":
    main_cli()

GraphRAG Question Answering System
Loading data from files...
  KG file: /kaggle/input/kltn-1/graph_documents_v3.json
  Questions file: /kaggle/input/kltn-1/question_1000.json
  Loaded 1644 entities
  Loaded 2662 triplets
  Loaded 356 multiple choice questions
  Loaded 724 true/false questions
GraphRAG Question Answering System
Device: CUDA
GPU Count: 2
  GPU 0: Tesla T4
  GPU 1: Tesla T4
Loading models...
LLM: deepseek-ai/DeepSeek-R1-0528-Qwen3-8B
Embedding: Qwen/Qwen3-Embedding-0.6B
Rerank: Qwen/Qwen3-Reranker-0.6B


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/859 [00:00<?, ?B/s]

Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}
2025-11-06 22:06:44.548358: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762466804.729230      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762466804.780304      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

  Progress: 710/724


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


  Progress: 720/724


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.



EVALUATION COMPLETED

                       FINAL RESULTS                        

MULTIPLE CHOICE: 192/356 = 53.9%
  Wrong: 157  No Answer: 5
TRUE/FALSE:      462/724 = 63.8%
  Wrong: 260  No Answer: 2

OVERALL:         654/1080 = 60.6%
TIME:            1476.55s

Peak GPU Memory: 15.63 GB
HTML report saved to: report.html

Evaluation complete!
